In [90]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
from tqdm import tqdm
import string
import numpy as np

In [91]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [92]:
punctuation_map = dict((ord(char), None) for char in string.punctuation)  #引入标点符号，为下步去除标点做准备

In [96]:
# Define the paragraph and words to generate embeddings for
df_paragraphs = pd.read_csv('/Users/carina/Downloads/courses/final thesis/dataset/annotated text.csv')

df_keywords_YAKE5 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_YAKE_5.csv')
df_keywords_YAKE5 = df_keywords_YAKE5.drop(df_keywords_YAKE5.columns[0],axis=1)

df_keywords_YAKE10 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_YAKE_10.csv')
df_keywords_YAKE10 = df_keywords_YAKE10.drop(df_keywords_YAKE10.columns[0],axis=1)

df_keywords_YAKE15 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_YAKE_15.csv')
df_keywords_YAKE15 = df_keywords_YAKE15.drop(df_keywords_YAKE15.columns[0],axis=1)

#df_cleanedtext = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/annotated_cleaned.csv')
#df_cleanedtext = df_cleanedtext.drop(df_cleanedtext.columns[0],axis=1)


In [97]:
df_emvecOfkw_YAKE5 = pd.DataFrame()
df_emvecOfkw_YAKE10 = pd.DataFrame()
df_emvecOfkw_YAKE15 = pd.DataFrame()
#df_emvecOfallwords = pd.DataFrame()

In [98]:
heads = df_paragraphs.columns.values.tolist()
#for i in tqdm(range(len(heads))):
for head in heads:
    j = 0
    pp = 0
    poorkeyword = 0
    #head = heads[i]
    paras = []
    paras = df_paragraphs[head].tolist()
    
    keyword_YAKE5 = []
    keyword_YAKE5 = df_keywords_YAKE5[head].tolist()
    
    words_embeddings = []
    for paragraphs,words in tqdm(zip(paras,keyword_YAKE5)):
        
        #paragraphs = paragraphs.lower()
        paragraphs = paragraphs.translate(punctuation_map)
        # Tokenize the paragraph and convert the tokens to IDs
        tokens = tokenizer.tokenize(paragraphs)
        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # Convert the input IDs to a PyTorch tensor
        input_ids = torch.tensor(input_ids).unsqueeze(0)  # Batch size 1

        # Generate embeddings for the input IDs using the BERT model
        outputs = model(input_ids)
        
        words = words.strip('[').strip(']')
        words = words.translate(punctuation_map)
        words = words.split(' ')
        
        word_embeddings = []
        
        for word in words:
            
            # Find the index of the first occurrence of the word in the tokens
            if word in tokens:
                index = tokens.index(word)
                # Extract the corresponding embedding vector from the BERT model output
                embedding = outputs.last_hidden_state[0][index].detach().numpy()
                word_embeddings.append(embedding)
            else :
                j = j + 1
                tem_token = tokenizer.tokenize(word)
                tem_em = []
                for item in tem_token:
                    if item in tokens:
                        index = tokens.index(item)
                        # Extract the corresponding embedding vector from the BERT model output
                        embedding = outputs.last_hidden_state[0][index].detach().numpy()
                        tem_em.append(embedding)
                    else:
                        pp += 1
                        #a = [0.0] * 768
                        #tem_em.append(a)
                if len(tem_em)!=0:
                    tem_em = np.mean(tem_em, axis=0)
                    word_embeddings.append(tem_em)
                else:
                    tem_em = [0.0] * 768
                    word_embeddings.append(tem_em)
                    poorkeyword += 1
                    print('terrible1')
        
        words_embeddings.append(word_embeddings)
        #print(words_embeddings)
        #print('one done')
    df_emvecOfkw_YAKE5[head] = words_embeddings
    print(j)
    print(pp)
    print(poorkeyword)

301it [02:47,  2.06it/s]

terrible1


303it [02:48,  1.81it/s]

terrible1


317it [02:56,  1.80it/s]

...

terrible1


335it [03:20,  1.73it/s]

terrible1


480it [04:35,  1.74it/s]

874
12
6


In [89]:
df_emvecOfkw_YAKE10.to_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/emvec_BERT_keywords_YAKE10.csv')

In [107]:
df_keywords_tfidf5 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_tfidf_5.csv')
df_keywords_tfidf5 = df_keywords_tfidf5.drop(df_keywords_tfidf5.columns[0],axis=1)

df_keywords_tfidf10 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_tfidf_10.csv')
df_keywords_tfidf10 = df_keywords_tfidf10.drop(df_keywords_tfidf10.columns[0],axis=1)

df_keywords_tfidf15 = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/keywords_tfidf_15.csv')
df_keywords_tfidf15 = df_keywords_tfidf15.drop(df_keywords_tfidf15.columns[0],axis=1)

df_cleanedtext = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/annotated_cleaned.csv')
df_cleanedtext = df_cleanedtext.drop(df_cleanedtext.columns[0],axis=1)

df_origintext = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/annotated_originalwords.csv')
df_origintext = df_origintext.drop(df_origintext.columns[0],axis=1)


In [108]:
df_emvecOfkw_tfidf5 = pd.DataFrame()
df_emvecOfkw_tfidf10 = pd.DataFrame()
df_emvecOfkw_tfidf15 = pd.DataFrame()

In [109]:
def get_all_index(lst=None, item=''):
    return [i for i in range(len(lst)) if lst[i] == item]

In [110]:
heads = df_paragraphs.columns.values.tolist()
#for i in tqdm(range(len(heads))):
for head in heads:
    j = 0
    pp = 0
    uselessoris = 0
    uselesskws = 0
    #head = heads[i]
    paras = []
    paras = df_paragraphs[head].tolist()
    
    keyword_tfidf10 = []
    keyword_tfidf10 = df_keywords_tfidf10[head].tolist()
    
    clean_word = []
    clean_word = df_cleanedtext[head].tolist()
    
    original_word = []
    original_word = df_origintext[head].tolist()
    
    words_embeddings = []
    #for paragraphs,words in tqdm(zip(paras,keyword_tfidf10)):
    for i in tqdm(range(len(paras))):
        
        paragraphs = paras[i]
        words = keyword_tfidf10[i]
        cleanword = clean_word[i].strip('[').strip(']')
        cleanword = cleanword.translate(punctuation_map)
        cleanword = cleanword.split(' ')
        #print(len(cleanword))
        
        originalword = original_word[i].strip('[').strip(']')
        originalword = originalword.translate(punctuation_map)
        originalword = originalword.split(' ')
        #print(len(originalword))
        
        #paragraphs = paragraphs.lower()
        paragraphs = paragraphs.translate(punctuation_map)
        # Tokenize the paragraph and convert the tokens to IDs
        tokens = tokenizer.tokenize(paragraphs)
        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # Convert the input IDs to a PyTorch tensor
        input_ids = torch.tensor(input_ids).unsqueeze(0)  # Batch size 1

        # Generate embeddings for the input IDs using the BERT model
        outputs = model(input_ids)
        
        words = words.strip('[').strip(']')
        words = words.translate(punctuation_map)
        words = words.split(' ')
        
        word_embeddings = []
        
        for word in words:
            
            # Find the index of the first occurrence of the word in the tokens
            if word in tokens:
                index = tokens.index(word)
                # Extract the corresponding embedding vector from the BERT model output
                embedding = outputs.last_hidden_state[0][index].detach().numpy()
                word_embeddings.append(embedding)
            else :
                j += 1
                #print(word)
                index = get_all_index(cleanword,word)
                #print(index)
                origins = []
                for ind in index:
                    origins.append(originalword[ind])
                #print(origins)    
                originem = []
                for ori in origins:
                    tem_token = tokenizer.tokenize(ori)
                    tem_em = []
                    for item in tem_token:
                        if item in tokens:
                            #print('found')
                            index1 = tokens.index(item)
                            # Extract the corresponding embedding vector from the BERT model output
                            embedding = outputs.last_hidden_state[0][index1].detach().numpy()
                            tem_em.append(embedding)
                        else:
                            #print('not found')
                            #print(ori)
                            #print(item)
                            pp += 1
                            #a = [0.0] * 768
                            #tem_em.append(a)
                            
                    if len(tem_em) != 0 :
                        tem_em = np.mean(tem_em, axis=0)
                        originem.append(tem_em)
                    else : 
                        uselessoris += 1
                        print('terrible0')
                
                if len(originem) != 0 :
                    
                    originem = np.mean(originem, axis=0)
                    word_embeddings.append(originem)
                else:
                    uselesskws += 1
                    originem = [0.0]*768
                    word_embeddings.append(originem)
                    print('terrible1')

        words_embeddings.append(word_embeddings)
        #print(words_embeddings)
        #print('one done')
    df_emvecOfkw_tfidf10[head] = words_embeddings
    print(j)
    print(pp)
    print(uselessoris)
    print(uselesskws)


  4%|█▉                                          | 21/480 [00:11<03:52,  1.97it/s]

terrible0
terrible0
terrible0
terrible1
terrible0
terrible0
terrible0
terrible1


 54%|███████████████████████                    | 258/480 [02:31<01:37,  2.28it/s]

terrible0
terrible1


100%|███████████████████████████████████████████| 480/480 [04:25<00:00,  1.81it/s]


...

terrible0
terrible1
terrible0
terrible1


 61%|██████████████████████████▏                | 293/480 [02:39<01:38,  1.91it/s]

terrible0
terrible0
terrible1


100%|███████████████████████████████████████████| 480/480 [04:08<00:00,  1.93it/s]

3103
9
4
3


In [78]:
df_emvecOfkw_tfidf5.to_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/emvec_BERT_keywords_tfidf5.csv')